# Q2 采购金额激增异常检测

依据PDF中的示例提示：
- 按供应商与月份汇总下单金额；
- 计算当月金额相对前三个月的均值的激增倍率（spike ratio）；
- 标记超过阈值（如≥3倍）的月份为异常；
- 可视化Top异常及按供应商的时间序列折线图。

为避免超时，开发阶段使用列裁剪与每供应商至多200条记录的子集进行。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

po = pd.read_excel(Path('/workspace/Sample Data.Case 1&2.xlsx'), sheet_name='Purchase Order', usecols=['新系统供应商编码','供应商名称\\直送供应商名称','提交时间','下单金额'])
po['提交时间'] = pd.to_datetime(po['提交时间'], errors='coerce')
po = po[po['提交时间'].notna()].copy()
po_small = po.sort_values('提交时间').groupby('新系统供应商编码').head(200)
po_small['_month'] = po_small['提交时间'].dt.to_period('M')

grp = po_small.groupby(['新系统供应商编码','_month'])['下单金额'].sum().reset_index()
grp = grp.sort_values(['新系统供应商编码','_month'])
grp['prev3_mean'] = grp.groupby('新系统供应商编码')['下单金额'].transform(lambda s: s.shift(1).rolling(3, min_periods=1).mean())
grp['spike_ratio'] = grp['下单金额'] / grp['prev3_mean']

anomalies = grp[(grp['prev3_mean']>0) & (grp['spike_ratio']>=3)].sort_values('spike_ratio', ascending=False)
print('子集异常月份数：', len(anomalies))
anomalies.head(10)

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(x='_month', y='spike_ratio', data=anomalies.head(10), color='firebrick')
plt.xticks(rotation=45, ha='right')
plt.title('采购金额激增Top10月份（子集）')
plt.ylabel('激增倍率（当月金额/前三月均值）')
plt.xlabel('月份')
plt.tight_layout()
plt.show()

### 时间序列示例（选取若干供应商）

In [ ]:
sample_vendors = anomalies['新系统供应商编码'].drop_duplicates().head(5)
plt.figure(figsize=(12,6))
for v in sample_vendors:
    s = grp[grp['新系统供应商编码']==v]
    plt.plot(s['_month'].astype(str), s['下单金额'], marker='o', label=v)
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.title('部分供应商月度下单金额时间序列（子集）')
plt.ylabel('下单金额')
plt.xlabel('月份')
plt.tight_layout()
plt.show()

### 结果与进一步分析建议
- 激增月份可能源于季节性、集中采购、项目单或异常操作（如价格异常、拆分订单），需结合合同、审批、收货与付款记录进行核查。
- 可在全量数据中输出异常清单并按供应商维度进行抽样穿透；
- 根据PDF提示，阈值可灵活调整（如≥3或更高），并对有明显季节性的供应商进行季节性分组处理以降低误报。

## 附加：全量运行与更细粒度分析（含品类维度）

In [ ]:
import pandas as pd
from pathlib import Path
outputs_path = Path('/workspace/KPMG_HW1/outputs')
outputs_path.mkdir(parents=True, exist_ok=True)

# 全量读取必要字段
po_full = pd.read_excel(Path('/workspace/Sample Data.Case 1&2.xlsx'), sheet_name='Purchase Order',                         usecols=['新系统供应商编码','供应商名称\直送供应商名称','提交时间','下单金额','一级分类','二级分类','三级分类'])
po_full['提交时间'] = pd.to_datetime(po_full['提交时间'], errors='coerce')
po_full = po_full[po_full['提交时间'].notna()].copy()
po_full['_month'] = po_full['提交时间'].dt.to_period('M')

# 月度聚合与spike计算
grp_full = po_full.groupby(['新系统供应商编码','_month'])['下单金额'].sum().reset_index()
grp_full = grp_full.sort_values(['新系统供应商编码','_month'])
grp_full['prev3_mean'] = grp_full.groupby('新系统供应商编码')['下单金额'].transform(lambda s: s.shift(1).rolling(3, min_periods=1).mean())
grp_full['spike_ratio'] = grp_full['下单金额'] / grp_full['prev3_mean']

anom_full_3x = grp_full[(grp_full['prev3_mean']>0) & (grp_full['spike_ratio']>=3)].sort_values('spike_ratio', ascending=False)
anom_full_5x = grp_full[(grp_full['prev3_mean']>0) & (grp_full['spike_ratio']>=5)].sort_values('spike_ratio', ascending=False)
print('全量异常月份数（≥3倍）：', len(anom_full_3x), '；（≥5倍）：', len(anom_full_5x))

# 品类维度：对异常月份，统计该月各品类的金额贡献
po_full_month = po_full.merge(grp_full[['新系统供应商编码','_month','prev3_mean','spike_ratio']], on=['新系统供应商编码','_month'], how='left')
po_anom = po_full_month[(po_full_month['prev3_mean']>0) & (po_full_month['spike_ratio']>=3)]
cat_contrib = po_anom.groupby(['新系统供应商编码','_month','一级分类'])['下单金额'].sum().reset_index()
cat_contrib['占比'] = cat_contrib.groupby(['新系统供应商编码','_month'])['下单金额'].transform(lambda s: s/s.sum())

# 导出CSV
anom_full_3x.to_csv(outputs_path/'Q2_spike_months_3x.csv', index=False)
anom_full_5x.to_csv(outputs_path/'Q2_spike_months_5x.csv', index=False)
cat_contrib.to_csv(outputs_path/'Q2_spike_category_contributions.csv', index=False)

# 可视化：Top供应商的异常月份spike曲线
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,6))
sample_vendors = anom_full_3x['新系统供应商编码'].drop_duplicates().head(5)
for v in sample_vendors:
    s = grp_full[grp_full['新系统供应商编码']==v]
    plt.plot(s['_month'].astype(str), s['spike_ratio'], marker='o', label=v)
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.title('部分供应商的spike_ratio时间序列（全量）')
plt.ylabel('spike_ratio')
plt.xlabel('月份')
plt.tight_layout()
plt.show()